## Building A Chatbot
In this video We'll go over an example of how to design and implement an LLM-powered chatbot. This chatbot will be able to have a conversation and remember previous interactions.

Note that this chatbot that we build will only use the language model to have a conversation. There are several other related concepts that you may be looking for:

- Conversational RAG: Enable a chatbot experience over an external source of data
- Agents: Build a chatbot that can take actions

This video tutorial will cover the basics which will be helpful for those two more advanced topics.

In [ ]:
import os
from dotenv import load_dotenv
load_dotenv() ## aloading all the environment variable

groq_api_key=os.getenv("GROQ_API_KEY")


In [3]:
from langchain_groq import ChatGroq
model=ChatGroq(model="Gemma2-9b-It",groq_api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001119B84AB30>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001119B878F70>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [5]:
from langchain_core.messages import HumanMessage

result=model.invoke([HumanMessage(content="Hi , My name is Rohit and I am a Chief AI Engineer")]).content


In [10]:
from langchain_core.messages import AIMessage
model.invoke(
    [
        HumanMessage(content="Hi , My name is Rohit and I am a Chief AI Engineer"),
        AIMessage(content="{result}"),
        HumanMessage(content="Hey What's my name and what do I do?")
    ]
)

AIMessage(content='You said your name is Rohit and you are a Chief AI Engineer.  \n\nIs there anything else I can help you with? 😊  \n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 31, 'prompt_tokens': 47, 'total_tokens': 78, 'completion_time': 0.056363636, 'prompt_time': 0.00163799, 'queue_time': 0.252242521, 'total_time': 0.058001626}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--96271088-b586-4ec1-b278-c7363f84b4d6-0', usage_metadata={'input_tokens': 47, 'output_tokens': 31, 'total_tokens': 78})

### Message History
We can use a Message History class to wrap our model and make it stateful. This will keep track of inputs and outputs of the model, and store them in some datastore. Future interactions will then load those messages and pass them into the chain as part of the input. Let's see how to use this!

In [11]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store={}

def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id]=ChatMessageHistory()
    return store[session_id]

with_message_history=RunnableWithMessageHistory(model,get_session_history)

In [12]:
config={"configurable":{"session_id":"chat1"}}

In [15]:
response=with_message_history.invoke(
    [HumanMessage(content="Hi , My name is Rohit and I am a Chief AI Engineer")],
    config=config
)

In [16]:
response.content

"Hi Rohit, it's great to meet you!\n\nBeing a Chief AI Engineer is a fascinating role. What are some of the most exciting AI projects you're working on these days?  I'm always eager to hear about innovative applications of AI. \n\n"

In [17]:
with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

AIMessage(content='Your name is Rohit.  \n\nHow can I help you today?  😄  \n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 160, 'total_tokens': 180, 'completion_time': 0.036363636, 'prompt_time': 0.0043088, 'queue_time': 0.25298822, 'total_time': 0.040672436}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--b1bbfb79-5b16-47b3-87b5-539081e10b70-0', usage_metadata={'input_tokens': 160, 'output_tokens': 20, 'total_tokens': 180})

In [18]:
## change the config-->session id
config1={"configurable":{"session_id":"chat2"}}
response=with_message_history.invoke(
    [HumanMessage(content="Whats my name")],
    config=config1
)
response.content

"As an AI, I have no memory of past conversations and do not know your name. If you'd like to tell me, I'd be happy to use it! 😊\n"

In [19]:
response=with_message_history.invoke(
    [HumanMessage(content="Hey My name is John")],
    config=config1
)
response.content

"Hi John! It's nice to meet you. What can I do for you today? 😊  \n\n"

In [20]:
response=with_message_history.invoke(
    [HumanMessage(content="Whats my name")],
    config=config1
)
response.content

'You told me your name is John!  😄  \n\nIs there anything else I can help you with?  \n'

### Prompt templates
Prompt Templates help to turn raw user information into a format that the LLM can work with. In this case, the raw user input is just a message, which we are passing to the LLM. Let's now make that a bit more complicated. First, let's add in a system message with some custom instructions (but still taking messages as input). Next, we'll add in more input besides just the messages.

In [24]:
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
prompt=ChatPromptTemplate.from_messages(
    [
        ("system","You are a helpful assistant.Amnswer all the question to the Best of your ability"),
        MessagesPlaceholder(variable_name="messages") # this is theslot in prompt where I want you to put all prior chat messages
    ]
)

chain=prompt|model

In [26]:
chain.invoke({"messages":[HumanMessage(content="Hi My name is Rohit")]})

AIMessage(content="Hello Rohit! It's nice to meet you. \n\nI'm ready to answer your questions to the best of my ability. What can I help you with today? 😊  \n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 41, 'prompt_tokens': 31, 'total_tokens': 72, 'completion_time': 0.074545455, 'prompt_time': 0.00171874, 'queue_time': 0.28593659, 'total_time': 0.076264195}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--3c70a4d9-7a63-450e-bdff-03136b126de7-0', usage_metadata={'input_tokens': 31, 'output_tokens': 41, 'total_tokens': 72})

In [27]:
with_message_history=RunnableWithMessageHistory(chain,get_session_history)

In [29]:
config = {"configurable": {"session_id": "chat3"}}
response=with_message_history.invoke(
    [HumanMessage(content="Hi My name is Rohit")],
    config=config
)

response.content

"Hi Rohit! \n\nIt's great to meet you too.  What can I do for you?  😄 \n"

In [30]:
response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

'Your name is Rohit!  😊  I remember that you told me. \n\nIs there anything else I can help you with? \n'

In [31]:
## Add more complexity

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

In [32]:
response=chain.invoke({"messages":[HumanMessage(content="Hi My name is Rohit")],"language":"Hindi"})
response.content

'नमस्ते रोहित! मुझे बताओ, मैं कैसे मदद कर सकता हूँ? \n'

Let's now wrap this more complicated chain in a Message History class. This time, because there are multiple keys in the input, we need to specify the correct key to use to save the chat history.

In [33]:
with_message_history=RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages"
)

In [34]:
config = {"configurable": {"session_id": "chat4"}}
repsonse=with_message_history.invoke(
    {'messages': [HumanMessage(content="Hi,I am Rohit")],"language":"Hindi"},
    config=config
)
repsonse.content

'नमस्ते रोहित! मैं आपकी मदद करने के लिए तैयार हूँ। आप मुझसे क्या पूछना चाहेंगे? 😊  \n'

In [35]:
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="whats my name?")], "language": "Hindi"},
    config=config,
)

In [36]:
response.content

'आपका नाम रोहित है। \n'

### Managing the Conversation History
One important concept to understand when building chatbots is how to manage conversation history. If left unmanaged, the list of messages will grow unbounded and potentially overflow the context window of the LLM. Therefore, it is important to add a step that limits the size of the messages you are passing in.
'trim_messages' helper to reduce how many messages we're sending to the model. The trimmer allows us to specify how many tokens we want to keep, along with other parameters like if we want to always keep the system message and whether to allow partial messages

In [49]:
from langchain_core.messages import SystemMessage,trim_messages
# trimmer=trim_messages(
#     max_tokens=25,
#     strategy="first",
#     token_counter=model,
#     include_system=True,
#     allow_partial=False,
#     # start_on="human"
# )

trimmer = trim_messages(
    max_tokens=25,
    strategy="last",
    token_counter=model,
    allow_partial=False
)

messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]
trimmer.invoke(messages)

[AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

In [40]:
from operator import itemgetter

from langchain_core.runnables import RunnablePassthrough

chain=(
    RunnablePassthrough.assign(messages=itemgetter("messages")|trimmer)
    | prompt
    | model
    
)

response=chain.invoke(
    {
    "messages":messages + [HumanMessage(content="What ice cream do i like")],
    "language":"English"
    }
)
response.content

"As an AI, I don't have access to your personal preferences like your favorite ice cream flavor.  \n\nWhat's your favorite ice cream? 😊🍦\n"

In [41]:
response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what math problem did i ask")],
        "language": "English",
    }
)
response.content

'You asked "whats 2 + 2" 😊  \n\n\n\n'

In [42]:
## Lets wrap this in the MEssage History
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)
config={"configurable":{"session_id":"chat5"}}

In [43]:
response = with_message_history.invoke(
    {
        "messages": messages + [HumanMessage(content="whats my name?")],
        "language": "English",
    },
    config=config,
)

response.content

"As a large language model, I don't have access to past conversations or any personal information about you, including your name.  \n\nIf you'd like to tell me your name, I'd be happy to know! 😊  \n\n"

In [44]:
response = with_message_history.invoke(
    {
        "messages": [HumanMessage(content="what math problem did i ask?")],
        "language": "English",
    },
    config=config,
)

response.content

"As a large language model, I have no memory of past conversations. If you'd like to ask me a math problem, I'm happy to help! 😊  Just let me know what it is. \n\n"